# Equivalent Circuit Model Identification

## Estimating ECM parameters from multi-pulse HPPC data

This notebook provides example usage for estimating stationary parameters for a two RC branch Thevenin model using multi-pulse HPPC data.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by upgrading some dependencies and importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
%pip install --upgrade openpyxl pandas -q

import numpy as np
import pandas as pd
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


## Setting up the model

In this example, we use the Thevenin model with two RC elements and the default parameter value for the "Open-circuit voltage [V]", as provided by the original PyBaMM class. The other relevant parameters for the ECM model implementation are updated as per the cell specification.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})
parameter_values = pybamm.ParameterValues("ECM_Example")
parameter_values.update(
    {
        "Cell capacity [A.h]": 3,
        "Nominal cell capacity [A.h]": 3,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "R2 [Ohm]": 2e-3,
        "C2 [F]": 3e4,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

## Importing data

Here we will use multiple HPPC pulses from an open dataset [1].

[1] Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1

The initial state can be either an initial SoC or initial open-circuit voltage. In this example, we get the initial OCV by accessing the voltage data.

In [ ]:
file_loc = r"../../data/Samsung_INR21700/multipulse_hppc.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

Let's take a look at the input current and output voltage.

In [ ]:
pybop.plot.trajectories(
    x=dataset["Time [s]"],
    y=dataset["Current function [A]"],
    title="Current vs Time",
    xaxis_title="Time / s",
    yaxis_title="Current / A",
);

In [ ]:
pybop.plot.trajectories(
    x=dataset["Time [s]"],
    y=dataset["Voltage [V]"],
    title="Voltage vs Time",
    xaxis_title="Time / s",
    yaxis_title="Voltage / V",
);

## Identifying the parameters

Now we can start the PyBOP fitting process by defining the parameters for identification. The initial guess for the resistance parameter is generated from a random sample of the prior distributions. These are influenced by the `r_guess` parameter below.

In [ ]:
r0_guess = 0.005
parameter_values.update(
    {
        "R0 [Ohm]": pybop.Parameter(
            pybop.Gaussian(r0_guess, r0_guess / 10, truncated_at=[0, 0.1]),
        ),
        "R1 [Ohm]": pybop.Parameter(
            pybop.Gaussian(r0_guess, r0_guess / 10, truncated_at=[0, 0.1]),
        ),
        "R2 [Ohm]": pybop.Parameter(
            pybop.Gaussian(r0_guess, r0_guess / 10, truncated_at=[0, 0.1]),
        ),
        "C1 [F]": pybop.Parameter(
            pybop.Gaussian(500, 100, truncated_at=[100, 1000]),
        ),
        "C2 [F]": pybop.Parameter(
            pybop.Gaussian(2000, 500, truncated_at=[1000, 10000]),
        ),
    }
)

The `Problem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function.

Initial state can be either "Initial SoC" or "Initial open-circuit voltage [V]". In this example, we get the initial OCV by accessing the voltage data, or we could pass a value, e.g., {"Initial open-circuit voltage [V]": 4.1}. Similarly, if SOC input is required, {"Initial SoC": 0.95}.

In [ ]:
parameter_values.set_initial_state(f"{df['Voltage'].to_numpy()[0]} V")
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
    solver=pybamm.CasadiSolver(mode="safe", dt_max=40),
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)

Next, we construct the optimisation class with our algorithm of choice and run it. For the sake of reducing the runtime of this example, we limit the maximum iterations; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=20,
    max_iterations=100,
)
optim = pybop.XNES(problem, options=options)
result = optim.run()

## Plotting and visualisation

Next, we use PyBOP's plotting utilities to visualise the results of the optimisation. This provides us with a visual confirmation of the optimiser's converged parameter values in the time-domain output.

In [ ]:
pybop.plot.problem(problem, inputs=result.best_inputs, title="Optimised Comparison");

### Convergence and parameter trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
result.plot_convergence()
result.plot_parameters();

## Concluding thoughts

This notebook illustrates how to perform parameter estimation for multi-pulse HPPC data, providing insights into the optimisation process through various visualisations.